Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/ml-frameworks/tensorflow/training/distributed-tensorflow-with-parameter-server/distributed-tensorflow-with-parameter-server.png)

# Distributed TensorFlow with parameter server
In this tutorial, you will train a TensorFlow model on the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset using native [distributed TensorFlow](https://www.tensorflow.org/deploy/distributed).

## Prerequisites
* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning (AML)
* If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [configuration notebook](../../../configuration.ipynb) to:
    * install the AML SDK
    * create a workspace and its configuration file (`config.json`)
* Review the [tutorial](../train-hyperparameter-tune-deploy-with-tensorflow/train-hyperparameter-tune-deploy-with-tensorflow.ipynb) on single-node TensorFlow training using the SDK

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-core 1.6.0.post1 (c:\python38\lib\site-packages), Requirement.parse('azureml-core~=0.1.0.15254362')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.6.0.post1 (c:\python38\lib\site-packages), Requirement.parse('azureml-core~=0.1.0.15254362')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.6.0.post1 (c:\python38\lib\site-packages), Requirement.parse('azureml-core~=0.1.0.15254362')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core.run:StepRun._from_dto with exception (azureml-core 1.6.0.post1 (c:\python38

## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [2]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


## Initialize workspace
Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [3]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
Workspace name: cmaksphillywestcentralus
Azure region: westcentralus
Subscription id: 06237282-1ddd-4677-9878-5bc19cc0001a
Resource group: ITPTestingWorkers


## Attach existing CMAKS compute
You will need to create a [CMAKS compute target](https://github.com/Azure/CMK8s-Samples/blob/master/docs/3.%20attach%20CMAKS%20compute.markdown) for training your model. 

In [8]:
# Choose a name for cmaks compute
# compute_name = <compute_name> # compute name
compute_name = 'cmakstestgpu2' # compute name

In [9]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.contrib.core.compute.cmakscompute import CmAksCompute

cmaks_compute = ComputeTarget(workspace=ws, name=compute_name)

## Train model on the CMKASK compute
Now that we have the compute target ready to go, let's run our distributed training job.

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

In [10]:
import os

project_folder = './tf-distr-ps'
os.makedirs(project_folder, exist_ok=True)

Copy the training script `tf_mnist_replica.py` into this project directory.

In [11]:
import shutil

shutil.copy('tf_mnist_replica.py', project_folder)

'./tf-distr-ps\\tf_mnist_replica.py'

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed TensorFlow tutorial. 

In [12]:
from azureml.core import Experiment

experiment_name = 'tf-distr-ps'
experiment = Experiment(ws, name=experiment_name)

### Create a TensorFlow estimator
The AML SDK's TensorFlow estimator enables you to easily submit TensorFlow training jobs for both single-node and distributed runs. For more information on the TensorFlow estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-tensorflow).

In [16]:
from azureml.train.dnn import TensorFlow, ParameterServer

script_params={
    '--num_gpus': 2, # define the gpu used by single node
    '--train_steps': 500
}

estimator = TensorFlow(source_directory=project_folder,
                       compute_target=cmaks_compute,
                       script_params=script_params,
                       entry_script='tf_mnist_replica.py',
                       node_count=2,
                       distributed_training=ParameterServer(worker_count=2),
                       use_gpu=True)

WARNING - framework_version is not specified, defaulting to version 1.13.


The above code specifies that we will run our training script on `2` nodes, with two workers and one parameter server. In order to execute a native distributed TensorFlow run, you must provide the argument `distributed_backend=ParameterServer()`. Using this estimator with these settings, TensorFlow and its dependencies will be installed for you. However, if your script also uses other packages, make sure to install them via the `TensorFlow` constructor's `pip_packages` or `conda_packages` parameters.

### Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [17]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: tf-distr-ps,
Id: tf-distr-ps_1591660653_c42089bf,
Type: azureml.scriptrun,
Status: Starting)


### Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

Alternatively, you can block until the script has completed training before running more code.

In [ ]:
run.wait_for_completion(show_output=True) # this provides a verbose log